In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt

In [20]:
dataset = pd.read_csv('input_data.csv')
#data_raw = dataset.drop(['Profession','Size of City'],axis=1)
X = dataset.iloc[:, 1:8].values
y = dataset.iloc[:, -1].values

In [21]:
dataset

,Instance,Year of Record,Gender,Age,Country,Size of City,Profession,University Degree,Wears Glasses,Hair Color,Body Height [cm],Income in EUR
0,1,1997.0,unknown,41.0,Belarus,1239930,STEEL,Bachelor,0,Blond,193,61031.94416
1,2,1996.0,other,41.0,Singapore,1603504,SAFE,Master,0,Black,186,91001.32764
2,3,2018.0,other,28.0,Norway,1298017,RECEIVABLES/PAYABLES,PhD,1,Brown,170,157982.17670
3,4,2006.0,other,33.0,Cuba,751903,FLEET,No,1,Black,171,45993.75793
4,5,2010.0,female,46.0,United Arab Emirates,95389,LEAD,No,0,Blond,188,38022.16217
...,...,...,...,...,...,...,...,...,...,...,...,...
185218,185219,2012.0,male,37.0,Norway,62975,PAYROLL,Bachelor,0,Brown,203,NaN
185219,185220,2016.0,other,42.0,Slovakia,46819,PURCHASING,No,0,Black,181,NaN
185220,185221,2019.0,other,50.0,Sweden,525072,NEIGBORHOOD,Master,1,Brown,162,NaN
185221,185222,1980.0,female,54.0,Netherlands,915892,IT,Master,1,Brown,162,NaN


In [4]:
z=X[0:111993,:]

In [5]:
z1=pd.DataFrame(z)

In [6]:
z1.tail()

,0,1,2,3,4,5,6
111988,1980,female,29,Honduras,205274,INDUSTRIAL,No
111989,2006,female,36,Tunisia,1012826,SENIOR,Master
111990,1993,female,36,Sri Lanka,60848,MESSENGER,Master
111991,2019,other,54,Denmark,848640,SENIOR,No
111992,2017,female,27,State of Palestine,325590,PROJECT,No


In [7]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values = np.nan, strategy='median') #control+i is used for inspection
imputer = imputer.fit(X[:, [0, 2]])
X[:, [0, 2]] = imputer.transform(X[:, [0, 2]])
df=pd.DataFrame(X)

In [8]:
X[pd.isnull(X)]  = 'NaN'
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_X_1 = LabelEncoder()
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1])
labelencoder_X_3 = LabelEncoder()
X[:, 3] = labelencoder_X_3.fit_transform(X[:, 3])
labelencoder_X_5 = LabelEncoder()
X[:, 5] = labelencoder_X_5.fit_transform(X[:, 5])
labelencoder_X_5 = LabelEncoder()
X[:, 6] = labelencoder_X_5.fit_transform(X[:, 6])
#labelencoder_X_5 = LabelEncoder()
#X[:, 8] = labelencoder_X_5.fit_transform(X[:, 8])

In [ ]:
onehotencoder = OneHotEncoder(categorical_features = [1, 3, 5])
X = onehotencoder.fit_transform(X).toarray()

In [9]:
split_data=X

In [10]:
X=X[0:111993,:]

In [11]:
y=y[0:111993]

In [12]:
y

array([ 61031.94416,  91001.32764, 157982.1767 , ..., 289951.3294 ,
       100046.5278 , 145886.2885 ])

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

In [14]:
import xgboost as xgb

In [15]:
d_train = xgb.DMatrix(X_train, y_train)
d_test = xgb.DMatrix(X_test)

In [16]:
 params_1 = {"objective":"reg:linear",
             'colsample_bytree': 1,
             'learning_rate': 0.15,
             'booster': 'gbtree',
             'base_score':0.5,
             'alpha': 5, 
             'max_depth' : 25, 
             'min_child_weight': 7, 
             'n_estimators' : 1000, 
             'Gamma': 4, 
             'Subsample' :0.6,  
             'early_stopping_rounds':10}

In [ ]:
ypred_list.append( bst.predict(d_test))

In [17]:
xgb_reg = xgb.train(params_1, d_train, num_boost_round=100)

[23:20:41] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [18]:
preds = xgb_reg.predict(d_test)

In [19]:
import math
math.sqrt(((preds-y_test)**2).mean())

76657.91217879039

In [ ]:
# DMatrix for k-fold cv
dmatrix_data = xgb.DMatrix(X, y)

In [ ]:
xg_reg = xgb.train(params=params_1, dtrain=dmatrix_data, num_boost_round=10)

In [ ]:
xgb.plot_importance(xg_reg)
plt.rcParams['figure.figsize'] = [500,500]
plt.show()

In [ ]:
params = {
    'objective': 'reg:linear',
    'colsample_bytree': 0.3,
    'learning_rate': 0.1,
    'max_depth': 5,
}
 
cross_val = xgb.cv(
    params=params,
    dtrain=dmatrix_data, 
    nfold=3,
    num_boost_round=50, 
    early_stopping_rounds=10, 
    metrics='error', 
    as_pandas=True, 
    seed=42)
 
print(cross_val.head())

In [ ]:
plt.figure(figsize=(50, 50))
xgb.plot_importance(xgb_reg)
plt.show()

In [ ]:
objective = "reg:linear"
seed = 100
n_estimators = 100
learning_rate = 0.1
gamma = 0.1
subsample = 0.8
colsample_bytree = 0.8
reg_alpha = 1
reg_lambda = 1
silent = False

parameters = {}
parameters['objective'] = objective
parameters['seed'] = seed
parameters['n_estimators'] = n_estimators
parameters['learning_rate'] = learning_rate
parameters['gamma'] = gamma
parameters['colsample_bytree'] = colsample_bytree
parameters['reg_alpha'] = reg_alpha
parameters['reg_lambda'] = reg_lambda
parameters['silent'] = silent

scores = []

cv_params = {'max_depth': [2,4,6,8],
             'min_child_weight': [1,3,5,7]
            }

gbm = GridSearchCV(xgb.XGBRegressor(
                                        objective = objective,
                                        seed = seed,
                                        n_estimators = n_estimators,
                                        learning_rate = learning_rate,
                                        gamma = gamma,
                                        subsample = subsample,
                                        colsample_bytree = colsample_bytree,
                                        reg_alpha = reg_alpha,
                                        reg_lambda = reg_lambda,
                                        silent = silent

                                    ),
                    
                    param_grid = cv_params,
                    iid = False,
                    scoring = "neg_mean_squared_error",
                    cv = 5,
                    verbose = True
)

gbm.fit(X_train,y_train)
print("Best parameters %s" %gbm.best_params_)
print("Best score %s" %gbm.best_score_)
slack_message("max_depth and min_child_weight parameters tuned! moving on to refinement", 'channel')

In [ ]:
X.shape

In [ ]:
z=pd.read_csv('tcd ml 2019-20 income prediction test (without labels).csv')

In [ ]:
k_new = pd.DataFrame(z)

In [ ]:
dt1=pd.DataFrame(z)

In [ ]:
dt2=dt1.drop(['Instance','Profession'],axis=1)

In [ ]:
dt3 = dt2.iloc[:, 0:6].values

In [ ]:
dt3.shape

In [ ]:
dt3=split_data[111993:,:]

In [ ]:
final_test = xgb.DMatrix(dt3)

In [ ]:
y_pred_final = xgb_reg.predict(final_test)

In [ ]:
Instance = k_new.Instance

In [ ]:
ans = np.stack((Instance,y_pred_final),axis= 1)

In [ ]:
ans

In [ ]:
ans = pd.DataFrame(ans)

In [ ]:
ans.to_csv('Submit_XGboost2.csv')

In [ ]:
evallist  = [(d_train,'train')]

ypred_list = []
